In [1]:
import psycopg2
import pandas as pd
import os
import urllib.request

In [2]:
connection = psycopg2.connect(database='mimic', user="mimicuser", password=os.environ["MIMICUSERPASSWORD"])
cursor = connection.cursor()
cursor.execute("set search_path to mimiciii")

In [3]:
# To get the list of subject_ids for whom waveform exists in mimic matched waveform database.

physionet_url = "https://physionet.org/files/mimic3wdb-matched/1.0/RECORDS"
data = urllib.request.urlopen(physionet_url)

subject_id_list = [l.decode("utf-8")[5:11] for l in data.readlines()]
df_matched = pd.DataFrame(subject_id_list, columns=['subject_id'])

In [4]:
query_subjectids_cohort = ("""
    SELECT  icu.subject_id, icu.icustay_id
    FROM    sepsis3_cohort coh, icustays icu
    WHERE   coh.icustay_id = icu.icustay_id AND 
            coh.excluded=0 
    """)
df_cohort = pd.read_sql(query_subjectids_cohort, connection)

In [5]:
df_matched

,subject_id
0,000020
1,000030
2,000033
3,000052
4,000079
...,...
10277,099955
10278,099982
10279,099983
10280,099992


In [6]:
df_waveform = pd.merge(df_matched.astype(int), df_cohort.astype(int), on='subject_id', how='inner')

In [7]:
query_update = """
        UPDATE sepsis3_cohort coh
        SET waveform_exists = 0"""
query_update_sepsis3_cohort = query_update
cursor.execute(query_update_sepsis3_cohort);

connection.commit();

In [8]:
# For all the subject_ids for which the waveform exists in mimic matched waveform database, set the value of column 'waveform_exists' to '1'
query_update = """
        UPDATE sepsis3_cohort coh
        SET waveform_exists = 1
        WHERE coh.icustay_id ={icustay_id}"""
for index, row in df_waveform.iterrows():
    query_update_sepsis3_cohort = query_update.format(icustay_id=str(row['icustay_id']))
    cursor.execute(query_update_sepsis3_cohort);

connection.commit();

In [9]:
connection.commit();

query_count_sepsis_waveform_exists = """
    SELECT  COUNT(*) 
    FROM    sepsis3_cohort coh 
    WHERE   coh.waveform_exists = 1"""
cursor.execute(query_count_sepsis_waveform_exists);
result = cursor.fetchall();
print(result)

[(4446,)]


In [10]:
query_all = """
    SELECT  * 
    FROM    sepsis3 coh 
    """
#cursor.execute(query_count_sepsis_waveform_exists);
result = pd.read_sql(query_all, connection);
result

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,...,sepsis_cdc_simple,elixhauser_hospital,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
0,200001,152234,1,2181-11-25 19:06:12,2181-11-28 20:59:25,metavision,2181-11-18 00:00:00,7.795972,BLOOD CULTURE,0.0,...,0,3,0,6,7,3,2,1.0,0.0,1.0
1,200003,163557,1,2199-08-02 19:50:04,2199-08-08 17:09:18,carevue,2199-08-02 00:00:00,0.826435,URINE,0.0,...,1,0,1,6,3,4,2,1.0,0.0,1.0
2,200006,189514,1,2159-09-03 11:28:14,2159-09-04 19:08:10,carevue,NaT,NaN,None,NaN,...,0,-5,1,1,2,2,2,1.0,0.0,1.0
3,200007,129310,1,2109-02-17 10:03:37,2109-02-18 17:03:12,carevue,NaT,NaN,None,NaN,...,0,0,0,1,2,2,2,1.0,0.0,1.0
4,200009,129607,1,2189-11-30 10:34:32,2189-12-02 14:17:37,carevue,2189-11-30 00:00:00,0.440648,None,NaN,...,0,-4,1,3,4,4,2,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,299993,169431,1,2149-11-13 21:01:05,2149-11-16 16:57:30,carevue,NaT,NaN,None,NaN,...,0,0,0,0,1,4,1,0.0,0.0,1.0
61528,299994,177406,1,2114-12-09 11:56:00,2114-12-15 20:29:14,carevue,2114-12-10 19:14:00,-1.304167,MRSA SCREEN,0.0,...,0,15,0,5,1,3,2,1.0,0.0,1.0
61529,299995,134959,1,2116-03-04 17:44:39,2116-03-06 18:03:55,carevue,2116-03-04 00:00:00,0.739340,SPUTUM,1.0,...,0,0,1,3,2,3,1,0.0,0.0,1.0
61530,299998,158288,0,2181-07-05 18:47:40,2181-07-07 17:24:38,metavision,2181-07-05 00:00:00,0.783102,MRSA SCREEN,0.0,...,1,8,1,4,5,1,1,1.0,0.0,0.0


In [6]:
result.columns.values

array(['icustay_id', 'hadm_id', 'excluded', 'intime', 'outtime',
       'dbsource', 'suspected_infection_time_poe',
       'suspected_infection_time_poe_days', 'specimen_poe',
       'positiveculture_poe', 'antibiotic_time_poe', 'blood_culture_time',
       'blood_culture_positive', 'age', 'gender', 'is_male', 'ethnicity',
       'race_white', 'race_black', 'race_hispanic', 'race_other',
       'metastatic_cancer', 'diabetes', 'height', 'weight', 'bmi',
       'first_service', 'hospital_expire_flag', 'thirtyday_expire_flag',
       'icu_los', 'hosp_los', 'sepsis_angus', 'sepsis_martin',
       'sepsis_explicit', 'septic_shock_explicit',
       'severe_sepsis_explicit', 'sepsis_nqf', 'sepsis_cdc',
       'sepsis_cdc_simple', 'elixhauser_hospital', 'vent', 'sofa', 'lods',
       'sirs', 'qsofa', 'qsofa_sysbp_score', 'qsofa_gcs_score',
       'qsofa_resprate_score'], dtype=object)

In [11]:
result[result['excluded']==0]

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,...,sepsis_cdc_simple,elixhauser_hospital,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
14,200021,109307,0,2114-12-26 19:45:12,2114-12-27 22:46:28,metavision,NaT,NaN,None,NaN,...,0,4,1,4,7,3,3,1.0,1.0,1.0
18,200028,181955,0,2133-10-29 17:13:50,2133-11-01 14:55:14,metavision,NaT,NaN,None,NaN,...,0,8,1,10,6,3,2,1.0,0.0,1.0
22,200033,198650,0,2198-08-07 17:56:17,2198-08-21 14:59:18,metavision,2198-08-08 01:00:00,-0.294248,MRSA SCREEN,0.0,...,1,5,0,1,1,2,0,0.0,0.0,0.0
43,200061,121149,0,2134-01-23 16:38:46,2134-01-25 16:59:14,metavision,2134-01-24 00:00:00,-0.306412,BLOOD CULTURE,0.0,...,0,6,0,2,2,4,1,0.0,0.0,1.0
54,200075,132255,0,2159-09-23 00:13:20,2159-09-25 01:55:17,metavision,2159-09-23 00:00:00,0.009259,MRSA SCREEN,0.0,...,1,10,1,8,4,3,2,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61498,299948,183226,0,2119-05-25 13:47:31,2119-05-27 12:31:54,metavision,2119-05-25 14:37:00,-0.034363,MRSA SCREEN,0.0,...,1,-4,0,3,2,2,2,1.0,0.0,1.0
61499,299949,199962,0,2118-07-16 05:42:45,2118-07-20 14:46:25,metavision,2118-07-16 00:00:00,0.238021,SWAB,0.0,...,0,-5,1,3,2,2,0,0.0,0.0,0.0
61500,299950,110990,0,2122-06-20 13:25:29,2122-06-22 18:21:02,metavision,2122-06-20 00:00:00,0.559363,MRSA SCREEN,0.0,...,1,4,0,5,2,2,2,1.0,0.0,1.0
61510,299962,146497,0,2195-11-25 19:04:06,2195-11-26 13:21:12,metavision,NaT,NaN,None,NaN,...,0,0,0,5,6,2,2,1.0,0.0,1.0
